In [1]:
from IPython.display import display
import joblib
import numpy as np
import pandas as pd
import sklearn.feature_selection
import sklearn.metrics
import sklearn.model_selection

import src
from run import filepath_portcalls_processed, filepath_inspections_processed, filepath_ships_classification, filepath_s, filepath_y

In [2]:
# Only ships not selected for mobility network.
portcalls = pd.read_pickle(filepath_portcalls_processed)
inspections = pd.read_pickle(filepath_inspections_processed)
expert_labels = portcalls.groupby('ship')['risk'].max()

# ALL ships - also selected ones for the mobility network!
all_ships = portcalls.ship.unique()
y_true_all = pd.Series({ship: inspections.get(ship, default=0) for ship in all_ships})
y_score_all = pd.Series({ship: expert_labels.get(ship) for ship in all_ships})
sensitive = portcalls.groupby('ship')['flag'].last().astype(int).astype(bool)
s_all = pd.Series({ship: sensitive.get(ship) for ship in all_ships})

# Ships used for classification
ships = np.load(filepath_ships_classification)
y_score = pd.Series({ship: expert_labels.get(ship) for ship in ships})
y_true = pd.read_pickle(filepath_y)
s = pd.read_pickle(filepath_s)
ys = y_true.values*3 + s

# Distribution targets on whole data

In [3]:
display(y_true_all.replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts().sort_index())
print()
display(y_true_all.replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index())

compliant                  6743
deficiencies              21088
detention+deficiencies     1631
dtype: int64

compliant                 0.228871
deficiencies              0.715769
detention+deficiencies    0.055359
dtype: float64

# Distribution target
## All data

In [4]:
data = {
    'sensitive': y_true_all[s_all].replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index(),
    'non-sensitive': y_true_all[~s_all].replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index(),
    'total': y_true_all.replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index()
}
pd.DataFrame(data)

,sensitive,non-sensitive,total
compliant,0.195664,0.231149,0.228871
deficiencies,0.554204,0.726851,0.715769
detention+deficiencies,0.250132,0.042001,0.055359


## Only data not used in global cargo ship network

In [5]:
data = {
    'sensitive':     y_true[s ].replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index(),
    'non-sensitive': y_true[~s].replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index(),
    'total':         y_true.replace({0: 'compliant', 1: 'deficiencies', 2: 'detention+deficiencies'}).value_counts(normalize=True).sort_index()
}
pd.DataFrame(data)

,sensitive,non-sensitive,total
compliant,0.172970,0.214599,0.211973
deficiencies,0.568506,0.742206,0.731250
detention+deficiencies,0.258524,0.043195,0.056776


# Distribution sensitive attribute

## All data

In [6]:
pd.Series(np.bincount(s_all) / len(s_all), index=['white flag', 'non-white flag'])

white flag        0.935816
non-white flag    0.064184
dtype: float64

## Only data not used in global cargo ship network

In [7]:
pd.Series(np.bincount(s) / len(s), index=['white flag', 'non-white flag'])

white flag        0.936928
non-white flag    0.063072
dtype: float64

# Performance

In [16]:
y_true

8606745    0
9741516    1
9374894    0
7212482    1
9475428    1
          ..
9495387    0
9404766    1
9422366    1
9689213    0
9630535    1
Length: 25574, dtype: int64

In [8]:
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
performance = [sklearn.metrics.roc_auc_score(y_true.iloc[test] > 0, y_score.iloc[test]) for _, test in skf.split(y_true, ys)]
print(f"AUC_y: {np.mean(performance):.3f}+-{np.std(performance):.3f}")

AUC_y: 0.543+-0.006


In [9]:
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
performance = [sklearn.metrics.roc_auc_score(s.iloc[test] > 0, y_score.iloc[test]) for _, test in skf.split(y_true, ys)]
print(f"AUC_y: {np.mean(performance):.3f}+-{np.std(performance):.4f}")

AUC_y: 0.672+-0.0097


# Confusion matrix

In [10]:
assert all(y_score.index == y_score.index)
confusion_matrix_sensitive = pd.DataFrame(
    data=sklearn.metrics.confusion_matrix(y_true[s], y_score[s]),
    index=pd.Index(['compliant', 'minor deficiencies', 'detention'], name='inspection result'),
    columns=pd.Index(['low', 'medium', 'high'], name='Expert label')
)
confusion_matrix_non_sensitive = pd.DataFrame(
    data=sklearn.metrics.confusion_matrix(y_true[~s], y_score[~s]),
    index=pd.Index(['compliant', 'minor deficiencies', 'detention'], name='inspection result'),
    columns=pd.Index(['low', 'medium', 'high'], name='Expert label')
)
pd.concat({'sensitive': confusion_matrix_sensitive, 'non sensitive': confusion_matrix_non_sensitive}, axis=1)

sensitive             non sensitive            
Expert label             low medium high           low medium high
inspection result                                                 
compliant                  1    261   17           319   4774   49
minor deficiencies         0    620  297           682  16370  732
detention                  0    150  267             1    788  246

In [11]:
(y_true > 0)[~s].value_counts()

True     18819
False     5142
dtype: int64

In [12]:
data = np.concatenate(
    (
        sklearn.metrics.confusion_matrix((y_true > 0)[s], y_score[s], normalize=None)[:2, :], 
        sklearn.metrics.confusion_matrix((y_true > 0)[~s], y_score[~s], normalize=None)[:2, :]
    ),
)

df = pd.DataFrame(
    data, 
    index=pd.MultiIndex.from_product([['Non-white', 'White'], ['Compliant', 'Non-compliant']], names=['Flag', 'True']),
    columns=['Low', 'Medium', 'High']
)
df = (
    df
    .reset_index()
    .melt(id_vars=['Flag', 'True'], var_name='Predicted', value_name='Number')
    .assign(Model='Baseline')
)
display(df)
df.to_pickle('models/confusion-matrix-baseline.pkl')

,Flag,True,Predicted,Number,Model
0,Non-white,Compliant,Low,1,Baseline
1,Non-white,Non-compliant,Low,0,Baseline
2,White,Compliant,Low,319,Baseline
3,White,Non-compliant,Low,683,Baseline
4,Non-white,Compliant,Medium,261,Baseline
5,Non-white,Non-compliant,Medium,770,Baseline
6,White,Compliant,Medium,4774,Baseline
7,White,Non-compliant,Medium,17158,Baseline
8,Non-white,Compliant,High,17,Baseline
9,Non-white,Non-compliant,High,564,Baseline


In [13]:
data = np.concatenate(
    (
        sklearn.metrics.confusion_matrix((y_true > 0)[s], y_score[s], normalize=None)[:2, :], 
        sklearn.metrics.confusion_matrix((y_true > 0)[~s], y_score[~s], normalize=None)[:2, :]
    ),
    axis=1
)

pd.DataFrame(
    data,
    index=pd.Index(['Compliant', 'Non-compliant'], name='True:'),
    columns=pd.MultiIndex.from_product([['Non-white', 'White'],['Low', 'Medium', 'High']], names=['Flag:', 'Predicted:']),
)

Flag:         Non-white             White            
Predicted:          Low Medium High   Low Medium High
True:                                                
Compliant             1    261   17   319   4774   49
Non-compliant         0    770  564   683  17158  978